# 최적 제어 아이디어

## 목적함수 정의
우리의 목표는 에너지를 절감하면서 세 실내기의 온도가 균형을 맞춰서 고루 떨어지고
최대한 빨리 목표 온도에 도달하게 하는것

### 기본 목적 함수
$$
J = w_1 \cdot \text{Var}(t_1, t_2, t_3) + w_2 \cdot \max(t_1, t_2, t_3) + w_3 \cdot E_{penalty}
$$

$\text{Var}(t_1, t_2, t_3)$는 각 실내기별 도달 시간의 분산을 의미하며 ex(5분, 10분, 40분이면 분산이 큼) 최소화하여 온도 불균형을 없애기 위한 항

$\max(t_1, t_2, t_3)$는 각 실내기의 도달 시간 중 가장 큰 값을 선택하는 것을 의미하며 최소화하여 가장 늦게 떨어지는 실내기도 빠른 시간 안에 도달하도록 유도하기위한 항

### 에너지 페널티 근사식
$$
E_{penalty} \approx \sum_{i=1}^{3} \left( w_F \cdot F_i + w_T \cdot (T_{bdy, i} - T_{set, i}) \right)
$$ 

### 최종 목적 함수 (부하 보상 포함)
$$
J_{final} = J - w_4 \sum_{i=1}^{3} \left( \frac{\tau_{real, i}}{\tau_{model, i}} - 1 \right) u_i
$$


각 실내기들의 상태 및 공통상태 액션을 인풋으로 받아서 각 실내기의 도달 시간을 예측하는 모델이 있다고 가정한다


실제 제어에서 실내기들의 상태와 공통 상태는 센서 데이터로 주어진다.

또한 가능한 액션 조합은 정해져있다. 
Tset을 2도로 binning
```
    풍량 1, 2, 3, 4 (4) 
    온도는 16, 18, 20, 22, 24 (5) 
    ------------------------------(4*5 = 20)
    OFF의 경우의 수 하나 (1) 
    ------------------------------(20+1 = 21)
    각 실내기마다니까.
    ------------------------------(21^3 = 9,261)
```

현재 관측된 s와 가능한 액션 조합에 대해서 모델을 이용해서 도달 시간을 구한다

-> 목적함수를 계산한다

-> 목적함수를 최소화하는 액션을 선택한다

-> 일정 주기(ex 5분)로 반복한다. 




## 부하 대응


모델의 시정수
$$
T(\hat{t}) = T_{goal} = T_{eq} + (T_0 - T_{eq}) e^{-\frac{\hat{t}}{\tau_{model}}}
$$

$$
% 시정수(tau)에 대해 정리한 수식
\tau_{model} = \frac{-\hat{t}}{\ln\left(\frac{T_{goal} - T_{eq}}{T_0 - T_{eq}}\right)}
$$

실측된 실제 시정수 
$$
T(n) = T_{eq} + (T_0 - T_{eq}) e^{-\frac{n}{\tau_{real}}}
$$

% 실제 시정수(tau_real) 역산 공식
$$
\tau_{real} = \frac{n}{\ln\left(\frac{T_0 - T_{eq}}{T(n) - T_{eq}}\right)}
$$ 


제어 입력 강도를 스칼라화 하기위한 u 정의
$$
u = \alpha \frac{F - F_{\min}}{F_{\max} - F_{\min}} + (1-\alpha) \frac{T_{set, \max} - T_{set}}{T_{set, \max} - T_{set, \min}}
$$


만약 부하가 크면 $\frac{\tau_{real, i}}{\tau_{model, i}}$ 은 1보다 크고 부하 보정 항은 음수가 된다 - > u를 크게 만들려고 한다

만약 부하가 없으면  $\frac{\tau_{real, i}}{\tau_{model, i}}$는 1이되고 부하 보정항은 0이되어 원래 목적함수를 따르게 된다

예측된 것보다 실제가 빠르게 떨어지면 $\frac{\tau_{real, i}}{\tau_{model, i}}$ 는 1보다 작게 되고 부하 보정항은 양수가 되어 u를 최소화하려고 한다


## classic MPC와의 비교

| 비교 항목 | 전통적 MPC (Classical MPC) | 제안하는 방식 (Proposed Approach) | 비고 |
| :--- | :--- | :--- | :--- |
| **모델 구조** | **State-Space Model (상태 공간 모델)**<br>($x_{t+1} = Ax_t + Bu_t$ 형태의 반복 연산) | **Learning-based Cooperative Reach-Time Model**<br>(학습 기반 협조 제어 및 도달 시간 모델)<br>(상태 $S$와 액션 $A$ 조합 $\rightarrow$ 도달 시간 $t$ 예측) | 3대 실내기의 복잡한 상호작용 및 다수 외생 변수($T_{od}$ 등)로 인한 **N-step 반복 예측의 오차 누적 문제 회피** |
| **예측 구간** | **Fixed Horizon ($N$ Step)**<br>(고정된 $N$ 스텝 앞까지만 예측) | **Flexible Goal Horizon**<br>(목표 온도 도달 시점까지를 한 번에 예측) |  |
| **최적화 기법** | **Quadratic Programming (QP)**<br>(연속 공간에서의 미분 기반 최적화) | **Finite Control Set (FCS-MPC)**<br>(이산 액션 공간 탐색을 통한 최적해 도출) | **Global Optimum 보장 및 Constraint Pruning (제약 조건 가지치기)**<br>※ 고부하 예측 시 풍량 하강/All OFF 등 비현실적 시나리오를 전문가 규칙(Expert Rule)으로 배제하여 탐색 효율 증대 |
| **적응성** | 복잡한 모델 파라미터 튜닝 필요 | **Physics-Based Compensation**<br>(물리적 시정수 $\tau$ 비율을 이용한 Cost 보정) | 모델 불일치(Model Mismatch)에 강건함 (Robust) |
